# Bilan Hydrique Climatologie

In [1]:
import numpy as np
import pandas as pd

# Définition de la station de référence
REF_STATION_NAME = 'La Petite Claye'
REF_STATION_LATLON = [48.541356, -1.615400]
# Altitude (m)
REF_STATION_ALTITUDE = 50.

# Identification de l'API Météo-France
APPLICATION_ID = 'ZlFGb1VCNzdlQ3c5QmhSMU1IbE8xQTluOE0wYTpUS3l1YkcweGJmSTJrQlJVaGNiSkNHTXczdHNh'

# Météo-France API
METEOFRANCE_API = 'DPClim'

# Départements où chercher des stations
ID_DEPARTEMENTS = [35, 50]

# Fréquence des données climatiques
METEOFRANCE_FREQUENCE = 'quotidienne'

# NN_NOMBRE = 20
# NN_RAYON_KM = None
NN_NOMBRE = None
NN_RAYON_KM = 35.

In [2]:
import meteofrance

# Initialisation d'un client pour accéder à l'API Météo-France
client = meteofrance.Client(APPLICATION_ID, METEOFRANCE_API)

# Issue a series of API requests an example. For use this test, you must first subscribe to the arome api with your application
client.session.headers.update({'Accept': '*/*'})

In [3]:
LECTURE_LISTE_STATIONS = False

l_listes = []
for id_dep in ID_DEPARTEMENTS:
    filepath_liste_stations = meteofrance.get_filepath_liste_stations(
        client, id_dep)
    
    if LECTURE_LISTE_STATIONS:
        # Lecture par département
        filepath_liste_stations = meteofrance.get_filepath_liste_stations(
            client, id_dep)
        df_liste_stations_dep = pd.read_csv(filepath_liste_stations, index_col=0)
    else:
        # Demande de la liste des stations pour le département
        section = meteofrance.SECTION_LISTE_STATIONS
        params = {'id-departement': id_dep}
        response = meteofrance.demande(
            client, section, params=params, frequence=METEOFRANCE_FREQUENCE)
        df_liste_stations_dep = meteofrance.response_text_to_frame(
            client, response, index_col=client.id_station_label)

        # Sauvegarde par département
        df_liste_stations_dep.to_csv(filepath_liste_stations)

    # Liste pour compilation
    l_listes.append(df_liste_stations_dep)

# Compilation
df_liste_stations_brute = pd.concat(l_listes, axis='index')

# Garder les stations valides seulement
df_liste_stations = meteofrance.filtrer_stations_valides(client, df_liste_stations_brute)

In [4]:
import geo

df_liste_stations_nn = geo.selection_plus_proches_voisins(
    df_liste_stations, REF_STATION_LATLON, client.latlon_labels,
    nombre=NN_NOMBRE, rayon_km=NN_RAYON_KM)

df_liste_stations_nn

/home/atantet/Documents/Travail/1_agri/agri-notebooks/geo.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_liste_stations_nn.loc[:, 'distance'] = dist_km


,nom,typePoste,lon,lat,alt,distance
id,,,,,,
35044001,BROUALAN,2,-1.640833,48.485667,99,6
50410003,PONTORSON,1,-1.505167,48.585667,33,9
35224001,PLERGUER,2,-1.843667,48.524833,30,17
35110003,FEINS SA,1,-1.596833,48.326833,87,24
35225001,PLESDER,2,-1.924833,48.406833,56,27
35178001,MEZIERES-SUR-C.,2,-1.439000,48.308833,71,29
50531001,ST OVIN,2,-1.248667,48.682500,155,31
50218001,GRANVILLE,3,-1.613667,48.834500,37,33
35228001,DINARD,0,-2.076333,48.584833,65,34


In [5]:
DATE_DEB_PERIODE = '2024-12-01T00:00:00Z'
DATE_FIN_PERIODE = '2024-12-31T23:00:00Z'

id_commandes = meteofrance.compiler_commandes_des_stations_periode(
    client, df_liste_stations_nn, DATE_DEB_PERIODE, DATE_FIN_PERIODE,
    frequence=METEOFRANCE_FREQUENCE)
id_commandes

{35044001: '789991692945',
 50410003: '789991693082',
 35224001: '789991693213',
 35110003: '789991693392',
 35225001: '789991693529',
 35178001: '789991693786',
 50531001: '789991694034',
 50218001: '789991695096',
 35228001: '789991695231'}

In [6]:
section = 'commande'
response = meteofrance.demande(client, section, {'id-cmde': 789991692945}, frequence='fichier')
response

<Response [201]>

In [ ]:
response